In [3]:
import pandas as pd
import random as rd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
rd.seed(6)

In [5]:
skip = sorted(rd.sample(xrange(1, 2197291), 2187291))

In [6]:
train = pd.read_csv("act_train.csv", skiprows=skip)

In [5]:
train.columns.values

array(['people_id', 'activity_id', 'date', 'activity_category', 'char_1',
       'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7',
       'char_8', 'char_9', 'char_10', 'outcome'], dtype=object)

In [ ]:
test = p

In [6]:
length = len(train)

train.apply(lambda v: sum(v.isnull())*1.0/length)

people_id            0.0000
activity_id          0.0000
date                 0.0000
activity_category    0.0000
char_1               0.9262
char_2               0.9262
char_3               0.9262
char_4               0.9262
char_5               0.9262
char_6               0.9262
char_7               0.9262
char_8               0.9262
char_9               0.9262
char_10              0.0738
outcome              0.0000
dtype: float64

In [7]:
people = pd.read_csv("people.csv")

people.columns.values

array(['people_id', 'char_1', 'group_1', 'char_2', 'date', 'char_3',
       'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 'char_9',
       'char_10', 'char_11', 'char_12', 'char_13', 'char_14', 'char_15',
       'char_16', 'char_17', 'char_18', 'char_19', 'char_20', 'char_21',
       'char_22', 'char_23', 'char_24', 'char_25', 'char_26', 'char_27',
       'char_28', 'char_29', 'char_30', 'char_31', 'char_32', 'char_33',
       'char_34', 'char_35', 'char_36', 'char_37', 'char_38'], dtype=object)

In [8]:
people.people_id.value_counts()[:5]

ppl_99042     1
ppl_333523    1
ppl_72603     1
ppl_17815     1
ppl_386880    1
Name: people_id, dtype: int64

In [9]:
train_merged = pd.merge(left=people, right=train, on="people_id", suffixes=["", "_act"])

train_merged.columns.values

array(['people_id', 'char_1', 'group_1', 'char_2', 'date', 'char_3',
       'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 'char_9',
       'char_10', 'char_11', 'char_12', 'char_13', 'char_14', 'char_15',
       'char_16', 'char_17', 'char_18', 'char_19', 'char_20', 'char_21',
       'char_22', 'char_23', 'char_24', 'char_25', 'char_26', 'char_27',
       'char_28', 'char_29', 'char_30', 'char_31', 'char_32', 'char_33',
       'char_34', 'char_35', 'char_36', 'char_37', 'char_38',
       'activity_id', 'date_act', 'activity_category', 'char_1_act',
       'char_2_act', 'char_3_act', 'char_4_act', 'char_5_act',
       'char_6_act', 'char_7_act', 'char_8_act', 'char_9_act',
       'char_10_act', 'outcome'], dtype=object)

In [10]:
train_merged_na = train_merged.apply(lambda v: sum(v.isnull())*1.0/length)
train_merged_na[train_merged_na>0]

char_1_act     0.9262
char_2_act     0.9262
char_3_act     0.9262
char_4_act     0.9262
char_5_act     0.9262
char_6_act     0.9262
char_7_act     0.9262
char_8_act     0.9262
char_9_act     0.9262
char_10_act    0.0738
dtype: float64

In [11]:
(pd.to_datetime(train_merged.date_act) - pd.to_datetime(train_merged.date)).sort_values()[:10]

2184   0 days
3064   0 days
5011   0 days
2358   0 days
9279   0 days
6395   0 days
1519   0 days
6936   0 days
6933   0 days
1515   0 days
dtype: timedelta64[ns]

In [12]:
train_merged.outcome.value_counts()

0    5547
1    4453
Name: outcome, dtype: int64

In [13]:
train_merged.iloc[:,0:3].head()

,people_id,char_1,group_1
0,ppl_100035,type 2,group 9439
1,ppl_100087,type 2,group 17304
2,ppl_100124,type 2,group 17304
3,ppl_100202,type 2,group 38766
4,ppl_100227,type 2,group 19768


In [14]:
(train_merged.outcome == 0).sum()/10000.0

0.55469999999999997

In [14]:
from sklearn.linear_model import LogisticRegression
from patsy import dmatrices

In [16]:
Y, X = dmatrices("outcome~char_38", data=train_merged, return_type="dataframe")
y = Y["outcome"]

In [17]:
X[:10]

,Intercept,char_38
0,1.0,100.0
1,1.0,44.0
2,1.0,30.0
3,1.0,89.0
4,1.0,91.0
5,1.0,91.0
6,1.0,90.0
7,1.0,90.0
8,1.0,72.0
9,1.0,0.0


In [18]:
model = LogisticRegression()
model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
import statsmodels.api as sm
from patsy import dmatrices

In [16]:
ols_formula = "C(outcome)~ C(activity_category), C(char_1), C(char_2), C(char_3), C(char_4), C(char_5), C(char_6), C(char_7), C(char_8), C(char_9), C(char_10)"

In [17]:
Y, X = dmatrices(ols_formula, data=train, return_type="dataframe")
y = Y.outcome
logistic_model = LogisticRegression()
model.fit(X, y)

PatsyError: Error converting data to categorical: observation with value <patsy.categorical._CategoricalBox object at 0x7f39904e2950> does not match any of the expected levels (expected: [<patsy.categorical._CategoricalBox object at 0x7f3990635d50>, <patsy.categorical._CategoricalBox object at 0x7f3990641410>, ..., <patsy.categorical._CategoricalBox object at 0x7f39906417d0>, <patsy.categorical._CategoricalBox object at 0x7f39906418d0>])
    C(outcome)~ C(activity_category), C(char_1), C(char_2), C(char_3), C(char_4), C(char_5), C(char_6), C(char_7), C(char_8), C(char_9), C(char_10)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [13]:
y, X = dmatrices(ols_formula, data=train, return_type="dataframe")
ols_model = sm.OLS(y, X)
ols_model.fit().summary()

PatsyError: Error interpreting categorical data: all items must be hashable
    outcome~ activity_category, char_1, char_2, char_3, char_4, char_5, char_6, char_7, char_8, char_9, char_10
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^